In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter, decimate, resample_poly, resample, wiener
from scipy.fft import fft
from IPython.display import Audio
%matplotlib qt

# AM Demodulation

## Mathematical Explanation

1. **AM Signal Representation:**
   - AM Signal: $s(t) = (A_c + m(t)) \cdot \cos(2\pi f_c t)$
     - Where $s(t)$ is the AM signal, $A_c$ is the amplitude of the carrier wave, $m(t)$ is the message signal, and $f_c$ is the carrier frequency.
2. **Envelope Detection:**
   - Simplest form of demodulation.
   - Rectify and smooth the AM signal.
   - Rectified Signal: $|s(t)| = |(A_c + m(t)) \cdot \cos(2\pi f_c t)|$
   - After smoothing and removing DC offset, the message signal $m(t)$ is recovered.
3. **Synchronous Detection:**
   - Involves mixing the AM signal with a local carrier of the same frequency and phase.
   - Demodulated Signal: $s(t) \cdot \cos(2\pi f_c t) = (A_c + m(t)) \cdot \cos^2(2\pi f_c t)$
   - Using trigonometric identities, the signal can be expressed as: $\frac{1}{2}(A_c + m(t)) + \frac{1}{2}(A_c + m(t))\cos(4\pi f_c t)$
   - This contains a low-frequency component (the desired message signal $m(t)$) and a high-frequency component (twice the carrier frequency).
   - A low-pass filter removes the high-frequency component, yielding: $\frac{1}{2}(A_c + m(t))$
   - Removing the DC offset, the original message signal $m(t)$ is recovered.

In [ ]:
signal = np.load('signal_-1.npy')
fs = 6.25E6  # Sample rate in Hz
t_end = signal.shape[0]/fs
t = np.arange(0, t_end, 1/fs)  # Time vector

**Low pass filter and decimation**

In [ ]:
b, a = np.load('dec_fil.npy')
signal_fil = lfilter(b, a, signal)
dec = 5
signal_dec = decimate(signal_fil, dec)
t_dec = t[::dec]
fs_dec = fs/dec

In [ ]:
plt.figure()
Pxx_dec, f = plt.psd(signal_dec, Fs=fs_dec, NFFT=int(fs_dec), scale_by_freq=False)
plt.show()

**Find N highest frequncies**

In [ ]:
N = 5
paired_freq_power = sorted(zip(f, Pxx_dec), key=lambda x: x[1], reverse=True)
top_N_frequencies = [freq for freq, power in paired_freq_power[:N]]
print(top_N_frequencies)

In [ ]:
selected_freq = None

**IF - Downconvert, Filtering, Decimation**

In [ ]:
IF_FREQ = 105E3
mixing_freq = selected_freq-IF_FREQ
signal_dncnvrt = np.cos(2*np.pi*mixing_freq*t_dec)*signal_dec
b, a = np.load('if_fil.npy')
signal_dncnvrt_iff = lfilter(b, a, signal_dncnvrt)

In [ ]:
dec_1 = 5
signal_dec_1 = decimate(signal_dncnvrt_iff, dec, n=2)
t_dec_1 = t_dec[::dec_1]
fs_dec_1 = fs_dec/dec_1

In [ ]:
plt.figure()
plt.psd(signal_dec_1, Fs=fs_dec_1, NFFT=int(fs_dec_1), scale_by_freq=False)
plt.show()

In [ ]:
# 1. Rectification
rectified_AM = np.abs(signal_dec_1)

# 2. Filtering
b, a = np.load('aud_fil.npy')

# Apply the filter to the rectified signal
filtered_message = filtfilt(b, a, rectified_AM)

# 3. DC Removal
# Assuming the message has zero DC component, the DC component in the rectified signal is approximately Ac.
# We subtract this to get the demodulated message signal
Ac = np.mean(filtered_message)
demodulated_message = filtered_message - Ac

In [ ]:
aud_fs = 44100
aud_sig = resample(demodulated_message, int(aud_fs*t_end))

In [ ]:
plt.figure()
plt.psd(aud_sig, Fs=aud_fs, NFFT=int(aud_fs), scale_by_freq=False)
plt.show()

In [ ]:
b, a = np.load('aud_fil_2.npy')
aud_sig_fil = filtfilt(b, a, aud_sig)

In [ ]:
plt.figure()
plt.psd(aud_sig_fil, Fs=aud_fs, NFFT=int(aud_fs/10), scale_by_freq=False)
plt.show()

In [ ]:
audio_player_dsp = Audio(aud_sig_fil, rate=44100)

audio_player_dsp

In [ ]:
aud_sig_fil_wiener = wiener(aud_sig_fil)

In [ ]:
audio_player_dsp_2 = Audio(aud_sig_fil_wiener, rate=44100)

audio_player_dsp_2

In [ ]:
# Estimate the noise spectrum (assuming the first 10% of aud_sig_fil is noise)
noise_samples = int(len(aud_sig_fil) * 0.1)
noise_spectrum = np.abs(fft(aud_sig_fil[:noise_samples]))

# Mean noise spectrum
mean_noise_spectrum = np.mean(noise_spectrum, axis=0)

# Perform FFT on the entire signal
signal_fft = fft(aud_sig_fil)
signal_magnitude = np.abs(signal_fft)
signal_phase = np.angle(signal_fft)

# Spectral subtraction
noise_reduction_factor = 0.1  # Adjust this factor to control the amount of noise reduction
subtracted_magnitude = signal_magnitude - noise_reduction_factor * mean_noise_spectrum
subtracted_magnitude[subtracted_magnitude < 0] = 0

# Reconstruct the signal
reconstructed_fft = subtracted_magnitude * np.exp(1j * signal_phase)
noise_reduced_aud_sig_fil = np.real(np.fft.ifft(reconstructed_fft))

In [ ]:
audio_player_dsp_3 = Audio(noise_reduced_aud_sig_fil, rate=44100)

audio_player_dsp_3